In [28]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import time

In [29]:
# 📂 1. Load ETTh2 Data from local file
file_path = "../data/raw/ETTh2.csv"

df_etth2 = pd.read_csv(file_path)

print("✅ ETTh2 dataset loaded from local file!")
print(df_etth2.head())


✅ ETTh2 dataset loaded from local file!
                  date       HUFL    HULL       MUFL   MULL   LUFL   LULL  \
0  2016-07-01 00:00:00  41.130001  12.481  36.535999  9.355  4.424  1.311   
1  2016-07-01 01:00:00  37.528000  10.136  33.936001  7.532  4.435  1.215   
2  2016-07-01 02:00:00  37.946999  11.309  35.330002  9.007  2.100  0.000   
3  2016-07-01 03:00:00  38.952000  11.895  35.543999  9.436  3.380  1.215   
4  2016-07-01 04:00:00  38.113998  11.476  35.410000  9.623  2.036  0.000   

          OT  
0  38.661999  
1  37.124001  
2  36.465000  
3  33.608501  
4  31.850500  


In [30]:
# 📜 2. Preprocessing
# ❗ IMPORTANT: These min and max values must be same as ETTh1 preprocessing min-max
# Update them according to your preprocessing!

file_path_etth1 = "../data/raw/ETTh1.csv"

df_etth1 = pd.read_csv(file_path_etth1)

# Get min and max for 'OT' column
min_OT = df_etth1['OT'].min()
max_OT = df_etth1['OT'].max()

df_etth2['OT'] = (df_etth2['OT'] - min_OT) / (max_OT - min_OT)

print("\n✅ Preprocessing applied (normalization)!")
print(df_etth2[['date', 'OT']].head())


✅ Preprocessing applied (normalization)!
                  date        OT
0  2016-07-01 00:00:00  0.853355
1  2016-07-01 01:00:00  0.822649
2  2016-07-01 02:00:00  0.809491
3  2016-07-01 03:00:00  0.752461
4  2016-07-01 04:00:00  0.717362


In [ ]:
# 🚀 FastAPI Server URL
fastapi_url = "http://localhost:8000/predict"

# 🧠 Rolling window for 96 sensor readings
rolling_window = []

# 📈 Stream row by row
for idx, row in df_etth2.iterrows():
    # Add new value to the window
    ot_value = row['OT']
    rolling_window.append(ot_value)

    # Keep only the last 96 points
    if len(rolling_window) > 96:
        rolling_window.pop(0)

    # Predict only when we have 96 points
    if len(rolling_window) == 96:
        # Convert rolling_window values to Python native types (e.g., float)
        payload = {
            "sensor_values": [float(value) for value in rolling_window]
        }

        try:
            response = requests.post(fastapi_url, json=payload)
            response.raise_for_status()
            preds = response.json()

            # 📋 Print results
            print("\n🎯 Prediction Results:")
            for model, pred in preds.items():
                print(f"{model}: {pred:.6f}")
         
            #split with print
            print("-------------------------------------------------")

            print(f"\n🕒 Time: {row['date']}")
            print(f"LSTM Prediction: {preds['LSTM_Prediction']:.6f}")
            print(f"PatchTST Prediction: {preds['PatchTST_Prediction']:.6f}")
            print(f"LLM Prediction: {preds['LLM_Prediction']:.6f}")
            print(f"Final Ensemble Prediction: {preds['Final_Ensembled_Prediction']:.6f}")

        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")

    # ⏳ Wait for 1 second to simulate real-time
    time.sleep(1)

print("\n✅ Test finished successfully!")